# TEXT MINING for PRACTICE: 그 외 텍스트 데이터 수집 방법
---

## 1. Speech to Text (STT)
- 음성을 텍스트로 바꾸는 머신러닝 모델로, 자막 생성등에 자주 쓰임 (유튜브).
- 또한 모든 음성 인식 어플리케이션들도 결국에는 텍스트 데이터를 기반으로 명령을 인식 받기 때문에, 우리가 흔히 말하는 AI에는 필수적인 단계
- 직접 학습을 시킬 수도 있지만, 아무리 최선을 다해서 학습시켜도 큰 기업의 API의 성능을 뛰어넘기란 쉽지 않음

### 1-1. 구글 API 활용

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade google-cloud-speech

In [ ]:
# 키 파일을 이렇게 전달
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "google_key.json" 

In [ ]:
#텍스트를 추출할 음성 파일
import IPython.display as ipd
ipd.Audio('resources/tajja_mono.wav') # load a local WAV file


In [ ]:
#https://cloud.google.com/speech-to-text/docs/quickstart-client-libraries
import io
import os

# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

# Instantiates a client
client = speech.SpeechClient()

# The name of the audio file to transcribe
file_name = os.path.join(
    'resources',
    'tajja_mono.wav')

# Loads the audio into memory
with io.open(file_name, 'rb') as audio_file:
    content = audio_file.read()
    audio = types.RecognitionAudio(content=content)

config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code='ko-KR')

# Detects speech in the audio file
response = client.recognize(config, audio)

for result in response.results:
    print('Transcript: {}'.format(result.alternatives[0].transcript))

### 1-2. NAVER Clova API활용
- 한국어의 경우에는 국내 API가 성능이 더 좋음

In [ ]:
CLIENT_ID = "YOUR_CLIENT_ID"
CLIENT_SECRET = "CLIENT_SECRET"

In [ ]:
# https://apidocs.ncloud.com/ko/ai-naver/clova_speech_recognition/stt/
import sys
import requests
client_id = CLIENT_ID
client_secret = CLIENT_SECRET
lang = "Kor" # 언어 코드 ( Kor, Jpn, Eng, Chn )
url = "https://naveropenapi.apigw.ntruss.com/recog/v1/stt?lang=" + lang
file_name = os.path.join(
    'resources',
    'tajja_mono.wav')
data = open(file_name, 'rb')
headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/octet-stream"
}
response = requests.post(url,  data=data, headers=headers)
rescode = response.status_code
if(rescode == 200):
    print (response.text)
else:
    print("Error : " + response.text)

## 2. Optical character recognition (OCR)
- OCR같은 경우는 Image recognition의 일종으로, 사람 (human)의 인식 능력과 큰 차이가 나지 않음
- 하지만 컴퓨터의 속도와 무한히 일을 시킬 수 있다는 점에서, 사람의 능력을 뛰어 넘었다고 볼 수 있음

### 2-1. Google Vision API 활용
- Image recognition task의 경우에는 상대적으로 음성에 비해 어느 정도 성능이 수렴되어 있기 때문에, 어떤 API를 써도 크게 상관없음
- 단, 손 글씨의 경우에는 차이가 있을 수 있음

In [ ]:
import sys
!{sys.executable} -m pip install google-cloud-vision

In [ ]:
#https://cloud.google.com/vision/docs/ocr#vision_text_detection-python

def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')

    for text in texts:
        print('\n"{}"'.format(text.description))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])

        print('bounds: {}'.format(','.join(vertices)))



![title](resources/mopobridge.jpg)

In [ ]:
detect_text("resources/mopobridge.jpg")

## 3. Text to Speech (TTS)
- Machine Learning 적용 이전 부터 존재했던 기술로, 위의 두 가지에 비해 상대적으로 어렵지 않음
- 단, 진짜 사람처럼 말하는 TTS를 만드는 것은 Machine Learning 기술에 의존 해야함 (예:GAN)

In [ ]:
import os
import sys
import urllib.request
client_id = CLIENT_ID
client_secret = CLIENT_SECRET
encText = urllib.parse.quote("이렇게 텍스트 마이닝을 사랑해주셔서 정말 감사드립니다.")
data = "speaker=mijin&speed=0&text=" + encText;
url = "https://naveropenapi.apigw.ntruss.com/voice/v1/tts"
request = urllib.request.Request(url)
request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
request.add_header("X-NCP-APIGW-API-KEY",client_secret)
response = urllib.request.urlopen(request, data=data.encode('utf-8'))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    with open('resources/result.mp3', 'wb') as f:
        f.write(response_body)
else:
    print("Error Code:" + rescode)

In [ ]:
#음성파일 확인
import IPython.display as ipd
ipd.Audio('resources/result.mp3') # load a local WAV file


## TASK. 한국어의 위대함을 극복해보자!

![title](resources/korean.jpg)

### Step 1. 이미지를 텍스트로!

In [ ]:
detect_text("resources/korean.jpg")

아직 크게 달라지지 않았습니다..

![title](resources/translate01.png)

### Step 2. 텍스트를 음성으로!

In [ ]:
import os
import sys
import urllib.request
client_id = CLIENT_ID
client_secret = CLIENT_SECRET
encText = urllib.parse.quote("""한꾹인똘만알아뽈쑤있께 짝썽하겠씁니다.
히까씨이케부쿠로역에썬
30초또안껄릴만끔가깝찌만쑥쏘까많이 낙후되어있꼬
엘베없꼬4층이라짐많으면깨꼬쌩합니따.
빠뀌뻘레나왔꼬 화짱씰 많이낡았씁니따.
끄래써 똥 역화짱씰까써 쌌씁니따
쩔때 여끼로오찌마쎄요!
뜨럽꼬 낡꼬 꼐딴을 좋아하씨는 뿐만 까쎄요!""")
data = "speaker=mijin&speed=0&text=" + encText;
url = "https://naveropenapi.apigw.ntruss.com/voice/v1/tts"
request = urllib.request.Request(url)
request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
request.add_header("X-NCP-APIGW-API-KEY",client_secret)
response = urllib.request.urlopen(request, data=data.encode('utf-8'))
rescode = response.getcode()
if(rescode==200):
    print("TTS mp3 저장")
    response_body = response.read()
    with open('resources/result_korean.mp3', 'wb') as f:
        f.write(response_body)
else:
    print("Error Code:" + rescode)

In [ ]:
#음성파일 확인
ipd.Audio('resources/result_korean.mp3') # load a local WAV file


### Step 3. 음성을 또 텍스트로!

In [ ]:
# https://apidocs.ncloud.com/ko/ai-naver/clova_speech_recognition/stt/
import sys
import requests
client_id = CLIENT_ID
client_secret = CLIENT_SECRET
lang = "Kor" # 언어 코드 ( Kor, Jpn, Eng, Chn )
url = "https://naveropenapi.apigw.ntruss.com/recog/v1/stt?lang=" + lang
file_name = os.path.join(
    'resources',
    'result_korean.mp3')
data = open(file_name, 'rb')
headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/octet-stream"
}
response = requests.post(url,  data=data, headers=headers)
rescode = response.status_code
if(rescode == 200):
    print (response.text)
else:
    print("Error : " + response.text)

완벽하지는 않지만, 이제 해석이 가능합니다!

![title](resources/translate02.png)

### Step 4. 다른 창의적인 방법 또 생각해보기!

![title](resources/korean2.jpeg)

![title](resources/korean3.jpg)